# Imports

In [1]:
%pip install -q evaluate
%pip install -q opendatasets
%pip install -q accelerate
%pip install -q --upgrade accelerate
%pip install -q transformers
%pip install -q --upgrade transformers
%pip install -q peft
%pip install -q --upgrade bitsandbytes
%pip install -q trl
%pip install -q nltk
%pip install -q -U nltk
%pip install -q rouge_score
%pip install -q bert_score
%pip install -q torch
%pip install -q -U scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you

In [3]:
import pandas as pd 
import torch
import torch.nn as nn
torch.cuda.set_per_process_memory_fraction(0.9)
torch.backends.cuda.matmul.allow_tf32 = True
import torchtext
from torch.utils.data import Dataset, random_split
from typing import List, Dict, Union
from typing import Any, TypeVar
import pandas as pd
import os
import copy
import gc
import evaluate
import opendatasets as od
from huggingface_hub import login
from typing import Optional, Tuple, Union
import statistics

from datasets import load_dataset, Features, Value
from datasets import Dataset
import accelerate

from peft import LoftQConfig, LoraConfig, get_peft_model, PeftModel

import transformers
from transformers.modeling_outputs import QuestionAnsweringModelOutput
from transformers import BertLMHeadModel, AutoConfig, BitsAndBytesConfig,Conv1D
from transformers import AutoTokenizer, Seq2SeqTrainingArguments 
from transformers import Seq2SeqTrainer, AutoModelForCausalLM, IntervalStrategy, AutoModelForQuestionAnswering
from transformers import TrainingArguments
from trl import SFTTrainer

from sklearn.model_selection import train_test_split

set a seed and confirm CUDA support

In [5]:
torch.manual_seed(2137)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.deterministic = True

print("PyTorch Version: ", torch.__version__)
print("torchtext Version: ", torchtext.__version__)
print(f"Using {'GPU' if str(DEVICE) == 'cuda' else 'CPU'}.")

PyTorch Version:  2.2.1+cu121
torchtext Version:  0.17.1+cpu
Using GPU.


# Dataset Download

## Downloading MedDialog Dataset

NOTE: you will need a kaggle API key for the following to work

In [ ]:
import json

# Path to JSON file
json_file_path = "kaggle.json"

# Open the file and read the content
try:
  with open(json_file_path, "r") as f:
    json_data = json.load(f)
except FileNotFoundError:
  print(f"Error: JSON file not found at {json_file_path}")
  exit(1)

# Access username and key from the JSON data
try:
  username = json_data["username"]
  key = json_data["key"]
except KeyError:
  print("Error: 'username' or 'key' key not found in JSON data")
  exit(1)

In [ ]:
os.environ['KAGGLE_USERNAME'] = username
os.environ['KAGGLE_KEY'] = key

# Assign the Kaggle data set URL into variable
dataset = 'https://www.kaggle.com/datasets/dsxavier/diagnoise-me'
# Using opendatasets let's download the data sets
od.download(dataset, "dataset")

## Downloading USMLE Dataset

In [3]:
USMLE_dataset = load_dataset("GBaker/MedQA-USMLE-4-options", split="test")

In [5]:
print(USMLE_dataset[0])
print(len(USMLE_dataset))

{'question': 'A junior orthopaedic surgery resident is completing a carpal tunnel repair with the department chairman as the attending physician. During the case, the resident inadvertently cuts a flexor tendon. The tendon is repaired without complication. The attending tells the resident that the patient will do fine, and there is no need to report this minor complication that will not harm the patient, as he does not want to make the patient worry unnecessarily. He tells the resident to leave this complication out of the operative report. Which of the following is the correct next action for the resident to take?', 'answer': 'Tell the attending that he cannot fail to disclose this mistake', 'options': {'A': 'Disclose the error to the patient and put it in the operative report', 'B': 'Tell the attending that he cannot fail to disclose this mistake', 'C': 'Report the physician to the ethics committee', 'D': 'Refuse to dictate the operative report'}, 'meta_info': 'step1', 'answer_idx': 

# Load Datasets

## Loading MedDialog Dataset

In [6]:
is_kaggle = (
    "KAGGLE_CLOUD" in os.environ or "KAGGLE_KERNEL_RUN_TYPE" in os.environ
)
if is_kaggle:
    DATA_PATH = "/kaggle/input/diagnoise-me/diagnose_en_dataset.feather"
else:
    DATA_PATH = "dataset\\diagnoise-me\\diagnose_en_dataset.feather"

SEQ_LEN: int = 1024
data = pd.read_feather(DATA_PATH)
SAMPLE_SIZE: int =  int(data.shape[0] * 0.05) #get 5% of the data
data = data[:SAMPLE_SIZE]
print(data.keys())
print(len(data))

Index(['id', 'Description', 'Doctor', 'Patient'], dtype='object')
3862


In [7]:
# Split data into train and eval sets with 70% for training
train_data, eval_data = train_test_split(data, test_size=0.3, random_state=42)

train_data = train_data.reset_index(drop=True)
eval_data = eval_data.reset_index(drop=True)

# Print the shapes of the train and eval sets
print("Train data shape:", train_data.shape)
print("Eval data shape:", eval_data.shape)

Train data shape: (2703, 4)
Eval data shape: (1159, 4)


## Loading USMLE Dataset

In [8]:
USMLE_dataset = pd.DataFrame({'Doctor': USMLE_dataset["answer"], 'Patient': USMLE_dataset["question"], 'Options':USMLE_dataset["options"]})
# Print the shapes of the set
print("USMLELiveEQA data shape:", USMLE_dataset.shape)

USMLELiveEQA data shape: (1273, 3)


## Create an output directory

In [9]:
os.makedirs('./results', exist_ok = True)
OUTPUT_DIR: str = './results'

# Model

In [10]:
# tokens for the datset
MODEL_NAME: str = 'UnfilteredAI/Mia-1B'

In [11]:
# Load tokenizer 
MAX_TOKEN_LENGTH = 1024

# for evaluation
ltokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
ltokenizer.padding_side = 'left'
ltokenizer.truncation_side = 'left'

# for training
rtokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
rtokenizer.padding_side = 'right'
rtokenizer.truncation_side = 'right'

In [12]:
base_model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
#base_model.resize_token_embeddings(len(rtokenizer))

In [13]:
print(base_model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Line

In [14]:
lora_config = LoraConfig(
    lora_alpha=16, # lora alpha for scaling
    r=16, # rank
    lora_dropout=0.05, #dropout
    use_rslora=True, #  sets the adapter scaling factor to lora_alpha/math.sqrt(r)
    bias="none", # dont train biases
    target_modules=["q_proj", "v_proj"],
    task_type="CAUSAL_LM",
    #layers_to_transform=[20]
)
# model = get_peft_model(base_model, lora_config)
# model.gradient_checkpointing_enable()
# model.enable_input_require_grads()

# Preparing Data for Training

## Custom Dataset

In [17]:
def build_dataset(data):
    listed_data = []
    try:
                listed_data = [[
                        {"role": "system", "content": "You are a medical professional providing consultation and medical diagnostics."},
                        {"role": "user", "content": f"{patient}, choose from A) {options['A']}, B) {options['B']}, C) {options['C']}, D)  {options['D']}"},
                        {"role": "assistant", "content": doctor}
                    ]for patient, doctor, options in zip(data["Patient"], data["Doctor"], data["Options"])]
    except:
                listed_data =  [[
                        {"role": "system", "content": "You are a medical professional providing consultation and medical diagnostics."},
                        {"role": "user", "content":patient},
                        {"role": "assistant", "content": doctor}
                    ]for patient, doctor in zip(data["Patient"], data["Doctor"])]
    dataset = {"messages": listed_data}
    dataset = Dataset.from_dict(dataset)
    return dataset



def build_prompted_dataset(data):
    listed_data = []
    try:
                listed_data = [[
                        {"role": "system", "content": "You are a medical professional providing consultation and medical diagnostics."},
                        {"role": "user", "content": f"A medical student is preparing for her final examination, in a practice question her patient says: '{patient}'. State the correct cause/course of action from the following options A) {options['A']}, B) {options['B']}, C) {options['C']}, D)  {options['D']}, and then provide an example explanation for the student"},
                        {"role": "assistant", "content": doctor}
                    ]for patient, doctor, options in zip(data["Patient"], data["Doctor"], data["Options"])]
    except:
                listed_data =  [[
                        {"role": "system", "content": "You are a medical professional providing consultation and medical diagnostics."},
                        {"role": "user", "content":f"A medical student is preparing for her final examination, in a practice question her patient says: '{patient}'. Provide a response to the patient as an example for the student"},
                        {"role": "assistant", "content": doctor}
                    ]for patient, doctor in zip(data["Patient"], data["Doctor"])]
    dataset = {"messages": listed_data}
    dataset = Dataset.from_dict(dataset)
    return dataset
                

In [18]:
train_dataset = build_dataset(train_data)
eval_dataset_1 = build_dataset(eval_data[0:5])
eval_dataset_2 = build_dataset(USMLE_dataset[0:5])

prompted_eval_dataset_1 = build_prompted_dataset(eval_data[0:5])
prompted_eval_dataset_2 = build_prompted_dataset(USMLE_dataset[0:5])

test_dataset = build_dataset(train_data[0:1])



## Custom Data Collator

In [20]:
def format_text(message, tokenizer, add_generation_prompt):
    text = tokenizer.apply_chat_template(
        message,
        tokenize=False,
        add_generation_prompt=add_generation_prompt
    )
    return text

def custom_data_collator(features, return_tensors="pt"):
    batch = {}

    tokenizer = ltokenizer

    messages = [feature['messages'][0:2] for feature in features]

    text = list(map(lambda x: format_text(x, tokenizer, True), messages))
    
    encoding = tokenizer(text, padding=True, max_length=MAX_TOKEN_LENGTH, return_tensors=return_tensors, add_special_tokens=True)


    # Prepare final batch dictionary
    batch["input_ids"] = encoding["input_ids"]
    batch["attention_mask"] = encoding["attention_mask"]

    return batch

# Training

In [ ]:
training_args = TrainingArguments(
    output_dir = OUTPUT_DIR, 
    num_train_epochs = 2, 
    evaluation_strategy="steps",
    #eval_steps = 50,
    #logging_steps = 50,
    save_total_limit = 1,
    per_device_train_batch_size=8, 
    per_device_eval_batch_size=1,
    bf16=False,
    fp16=True,
    warmup_steps=0, 
    weight_decay=0.01, 
    logging_dir='./logs',
    save_steps = 0,
    load_best_model_at_end=True,
    eval_accumulation_steps=10,
    report_to=['tensorboard']
    )

In [ ]:
trainer = SFTTrainer(
    model=base_model, 
    args=training_args, 
    peft_config=lora_config,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset_1, 
    max_seq_length=1024,
    packing=False
)
trainer.model.gradient_checkpointing_enable()
trainer.model.enable_input_require_grads()

In [ ]:
model.config.pad_token_id = ltokenizer.pad_token_id

In [ ]:
print(model)

In [ ]:
# clear memory
# you may find value in running this cell twice
torch.cuda.empty_cache()
gc.collect()

In [ ]:
# train the model
trainer.train()

In [ ]:
# save the model locally
trainer.model.save_pretrained(f"{OUTPUT_DIR}/model_save")

In [ ]:
# this is for pushing the trained model to the hub, please ignore this
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# please ignore this
tok = AutoTokenizer.from_pretrained(MODEL_NAME)
tok.push_to_hub("SurtMcGert/advanced-AI-CW-Med-Chat-Bot")

In [ ]:
# please ignore this
trainer.model.push_to_hub("SurtMcGert/advanced-AI-CW-Med-Chat-Bot")

In [ ]:
# clear memory
trainer = None
model = None
torch.cuda.empty_cache()
gc.collect()

# Load the Model

In [21]:
# define the arguments for the Seq2Seq trainer that we use for evaluation
eval_args = Seq2SeqTrainingArguments(
    output_dir = OUTPUT_DIR, 
    num_train_epochs = 1, 
    evaluation_strategy="steps",
    save_total_limit = 1,
    per_device_train_batch_size=8, 
    per_device_eval_batch_size=1,
    bf16=False,
    fp16=True,
    warmup_steps=0, 
    weight_decay=0.01, 
    logging_dir='./logs',
    save_steps = 0,
    load_best_model_at_end=True,
    remove_unused_columns=False,
    generation_config=transformers.GenerationConfig(
            max_new_tokens=100, # the max number of new tokens to generate
            early_stopping=True, # stop when unlikely to find better candidates
            # repetition_penalty = 1.5, # ads a penalty for repetition
            num_beams=25, # num of beams
            num_beam_groups=1, # num of beam groups
            do_sample=True, # use sampling instead of greedy search
            temperature=0.5, # modulates the next token probabilities
            diversity_penalty=0.0, # ads a penalty for generating unoriginal tokens
            encoder_repetition_penalty=0.99, # ads a penalty for producing tokens that are not in the original input
            no_repeat_ngram_size=5, # all ngrams of this size can only occur once
            # guidance_scale = 1, # Higher guidance scale encourages the model to generate samples that are more closely linked to the input prompt, usually at the expense of poorer quality.
            # length_penalty=-2 # promotes shorter token sequences
    ),
    predict_with_generate=True,
    eval_accumulation_steps=10,
    report_to=['none']
    )

In [22]:
model = AutoModelForCausalLM.from_pretrained("SurtMcGert/advanced-AI-CW-Med-Chat-Bot", force_download=True).to(DEVICE)
model.config.pad_token_id = ltokenizer.pad_token_id
model.config.max_length = MAX_TOKEN_LENGTH

# define the trainer
evaluator = Seq2SeqTrainer(
    model=model, 
    args=eval_args, 
    train_dataset=train_dataset,
    eval_dataset=eval_dataset_1, 
    data_collator=custom_data_collator
)

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

C:\Users\harry\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\harry\.cache\huggingface\hub\models--UnfilteredAI--Mia-1B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/145 [00:00<?, ?B/s]

In [24]:
# clear memory again, run this more than once if needs be
torch.cuda.empty_cache()
gc.collect()

0

# Evaluation

In [310]:
# predict responses on the MedDialog dataset
eval_result_1 = evaluator.predict(eval_dataset_1)
prompted_eval_result_1 = evaluator.predict(prompted_eval_dataset_1)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [294]:
# predict responses on the USMLE dataset
eval_result_2 = evaluator.predict(eval_dataset_2)
prompted_eval_result_2 = evaluator.predict(prompted_eval_dataset_2)

In [311]:
# get the logits for the predictions
logits_1 = eval_result_1.predictions
logits_1[logits_1 == -100] = ltokenizer.eos_token_id
logits_2 = eval_result_2.predictions
logits_2[logits_2 == -100] = ltokenizer.eos_token_id

prompted_logits_1 = prompted_eval_result_1.predictions
prompted_logits_1[prompted_logits_1 == -100] = ltokenizer.eos_token_id
prompted_logits_2 = prompted_eval_result_2.predictions
prompted_logits_2[prompted_logits_2 == -100] = ltokenizer.eos_token_id

In [312]:
# get the raw evaluation output
raw_text_result_1 = ltokenizer.batch_decode(logits_1, skip_special_tokens=True)
raw_text_result_2 = ltokenizer.batch_decode(logits_2, skip_special_tokens=True)

prompted_raw_text_result_1 = ltokenizer.batch_decode(prompted_logits_1, skip_special_tokens=True)
prompted_raw_text_result_2 = ltokenizer.batch_decode(prompted_logits_2, skip_special_tokens=True)

In [313]:
# get the questions and ground truths from both evaluation datasets
questions_1 = []
ground_truth_1 = []
try:
    for item in eval_dataset_1['messages']:
        questions_1.append(item[1]["content"])
        ground_truth_1.append(item[2]["content"])
except:
    pass

questions_2 = []
ground_truth_2 = []
try:
    for item in eval_dataset_2['messages']:
        questions_2.append(item[1]["content"])
        ground_truth_2.append(item[2]["content"])
except:
    pass

# create lists for the text outputs
text_result_1 = list()
text_result_2 = list()

prompted_text_result_1 = list()
prompted_text_result_2 = list()

# get the answers for the MedDialog dataset
for item in raw_text_result_1:
    index = item.find("<|assistant|>")
    output = item[index+13:]
    index = output.find(ltokenizer.eos_token)
    if(index > -1):
        output = output[:index]
    text_result_1.append(output)


# get the answers for the USMLE dataset
for item in raw_text_result_2:
    index = item.find("<|assistant|>")
    output = item[index+13:]
    index = output.find(ltokenizer.eos_token)
    if(index > -1):
        output = output[:index]
    text_result_2.append(output)


# get the answers for the MedDialog dataset
for item in prompted_raw_text_result_1:
    index = item.find("<|assistant|>")
    output = item[index+13:]
    index = output.find(ltokenizer.eos_token)
    if(index > -1):
        output = output[:index]
    prompted_text_result_1.append(output)


# get the answers for the USMLE dataset
for item in prompted_raw_text_result_2:
    index = item.find("<|assistant|>")
    output = item[index+13:]
    index = output.find(ltokenizer.eos_token)
    if(index > -1):
        output = output[:index]
    prompted_text_result_2.append(output)
    
    
# save to csv
data = {"question": questions_1, "ground_truth": ground_truth_1, "prediction": text_result_1, "prompted_prediction": prompted_text_result_1}
df = pd.DataFrame(data)
df.to_csv("eval_output_1.csv", index=False) 
data = {"question": questions_2, "ground_truth": ground_truth_2, "prediction": text_result_2, "prompted_prediction": prompted_text_result_2}
df = pd.DataFrame(data)
df.to_csv("eval_output_2.csv", index=False)  



# print the first 2 results from each dataset evaluation
print("============================MedDialog Evaluation============================")
for question, gt, answer, prompted_answer in list(zip(questions_1, ground_truth_1, text_result_1, prompted_text_result_1))[:2]:
    print(f"""
    Question: {question}
    Ground Truth: {gt}
    Prediction: {answer}
    Prompted_prediction: {prompted_answer}
    """)

print("============================USMLE Evaluation============================")
for question, gt, answer, prompted_answer in list(zip(questions_2, ground_truth_2, text_result_2, prompted_text_result_2))[:2]:
    print(f"""
    Question: {question}
    Ground Truth: {gt}
    Prediction: {answer}
    Prompted_prediction: {prompted_answer}
    """)

============================MedDialog Evaluation============================

    Question: Hi doctor, I am a 15 year old boy and I have hyperhidrosis. I sweat so much. My shirt cannot stay on for at least 5 minutes. In some half an hour to one hour my whole shirt and armpit area will drench in sweat. Yes, I take a shower twice a day and put deodorant every day. I have even tried the antiperspirant deodorant, but that did not stop it and I think it made it worse. I really want to believe it is just puberty as this has been going on since I was 12 or 13. But, something tells me that it is not because of puberty. Everyone else at school does not have this problem and some relatives in my family has it too. Just like my shirt, my pants will start soaking after 30 minutes of sitting down. I really want a diagnosis or close to one before I actually go to a doctor as I have not yet regarding my issue. Also are there any home remedies or other over-the-counter deodorant to help minimize this 

### Load The Evaluations

In [30]:
# this just loads the data from the two CSV files, you can ignore this step please
file_path = "eval_output_1.csv"

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)

# Extract data into separate variables
questions_1 = df['question'].tolist()
ground_truth_1 = df['ground_truth'].tolist()
text_result_1 = df['prediction'].tolist()
prompted_text_result_1 = df['prompted_prediction'].tolist()


file_path = "eval_output_2.csv"

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)

# Extract data into separate variables
questions_2 = df['question'].tolist()
ground_truth_2 = df['ground_truth'].tolist()
text_result_2 = df['prediction'].tolist()
prompted_text_result_2 = df['prompted_prediction'].tolist()


# Results

## Load the Required Evaluation Metrics

In [298]:
# perplexity - measures certainty of the model.
# METEOR - extension of BLEU (measure similarity between the output and the ground truth) but accounts for word semantics.
# ROUGE - considers n-gram overlap (recall) but also precision.
# Accuracy - use this for the multiple choice dataset

perplexity_scorer = evaluate.load('perplexity')
meteor_scorer = evaluate.load('meteor')
rouge_scorer = evaluate.load('rouge')
accuracy_scorer = evaluate.load('accuracy')
bert_scorer = evaluate.load("bertscore")


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [299]:
# compute the scores for the MedDialog evaluation
perplexity_score_1 = perplexity_scorer.compute(model_id='gpt2', predictions=text_result_1, references=ground_truth_1)
meteor_score_1 = meteor_scorer.compute(predictions=text_result_1, references=ground_truth_1)
rouge_score_1 = rouge_scorer.compute(predictions=text_result_1, references=ground_truth_1)
bert_score_1 = bert_scorer.compute(predictions=text_result_1, references=ground_truth_1, lang="en")
bert_score_1['precision'] = statistics.mean(bert_score_1['precision'])
bert_score_1['recall'] = statistics.mean(bert_score_1['recall'])
bert_score_1['f1'] = statistics.mean(bert_score_1['f1'])

prompted_perplexity_score_1 = perplexity_scorer.compute(model_id='gpt2', predictions=prompted_text_result_1, references=ground_truth_1)
prompted_meteor_score_1 = meteor_scorer.compute(predictions=prompted_text_result_1, references=ground_truth_1)
prompted_rouge_score_1 = rouge_scorer.compute(predictions=prompted_text_result_1, references=ground_truth_1)
prompted_bert_score_1 = bert_scorer.compute(predictions=prompted_text_result_1, references=ground_truth_1, lang="en")
prompted_bert_score_1['precision'] = statistics.mean(bert_score_1['precision'])
prompted_bert_score_1['recall'] = statistics.mean(bert_score_1['recall'])
prompted_bert_score_1['f1'] = statistics.mean(bert_score_1['f1'])

  0%|          | 0/1 [00:00<?, ?it/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# compute the scores for the USMLE evaluation
perplexity_score_2 = perplexity_scorer.compute(model_id='gpt2', predictions=text_result_2, references=ground_truth_2)
meteor_score_2 = meteor_scorer.compute(predictions=text_result_2, references=ground_truth_2)
rouge_score_2 = rouge_scorer.compute(predictions=text_result_2, references=ground_truth_2)
bert_score_2 = bert_scorer.compute(predictions=text_result_2, references=ground_truth_2, lang="en")
bert_score_2['precision'] = statistics.mean(bert_score_2['precision'])
bert_score_2['recall'] = statistics.mean(bert_score_2['recall'])
bert_score_2['f1'] = statistics.mean(bert_score_2['f1'])


prompted_perplexity_score_2 = perplexity_scorer.compute(model_id='gpt2', predictions=prompted_text_result_2, references=ground_truth_2)
prompted_meteor_score_2 = meteor_scorer.compute(predictions=prompted_text_result_2, references=ground_truth_2)
prompted_rouge_score_2 = rouge_scorer.compute(predictions=prompted_text_result_2, references=ground_truth_2)
prompted_bert_score_2 = bert_scorer.compute(predictions=prompted_text_result_2, references=ground_truth_2, lang="en")
prompted_bert_score_2['precision'] = statistics.mean(bert_score_2['precision'])
prompted_bert_score_2['recall'] = statistics.mean(bert_score_2['recall'])
prompted_bert_score_2['f1'] = statistics.mean(bert_score_2['f1'])

In [288]:
# print scores for MedDialog evaluation
print("========= NO PROMPT ENGINEERING =========")
print(perplexity_score_1)
print(meteor_score_1)
print(rouge_score_1)
print(bert_score_1)

print("========= PROMPT ENGINEERING =========")
print(prompted_perplexity_score_1)
print(prompted_meteor_score_1)
print(prompted_rouge_score_1)
print(prompted_bert_score_1)

{'perplexities': [7.651326656341553, 16.207849502563477, 11.66840648651123, 34.25922393798828, 9.206561088562012], 'mean_perplexity': 15.798673534393311}
{'meteor': 0.2993209924843062}
{'rouge1': 0.2441136701249123, 'rouge2': 0.13138005938866157, 'rougeL': 0.18608098627827493, 'rougeLsum': 0.17461719690750882}
{'precision': 0.7922456979751586, 'recall': 0.8621158838272095, 'f1': 0.8243112325668335, 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.40.1)'}


In [ ]:
# print scores for USMLE evaluation
print("========= NO PROMPT ENGINEERING =========")
print(perplexity_score_2)
print(meteor_score_2)
print(rouge_score_2)
print(bert_score_2)

print("========= PROMPT ENGINEERING =========")
print(prompted_perplexity_score_2)
print(prompted_meteor_score_2)
print(prompted_rouge_score_2)
print(prompted_bert_score_2)

# Interactive Chat

## Example Data Extract To Use

I am a 39-year-old woman. I have mild pain in the left side of the chest (below the neck and the breast) and then sensation in the upper back for four days. It comes and goes. Sometimes it goes to the right side of the chest also. I had my ECG and blood test 6 months ago. ECG and blood sugar were normal. No hypertension but hemoglobin was 10 and Vitamin B 12 was below average. What can I do as in lockdown it is not possible to see the doctor as a person. Please help.

In [21]:
# clear memory
model = None
base_model = None
trainer = None
evaluator = None
torch.cuda.empty_cache()
gc.collect()

0

In [10]:
# load the model
model = AutoModelForCausalLM.from_pretrained("SurtMcGert/advanced-AI-CW-Med-Chat-Bot").to(DEVICE)
tokenizer = AutoTokenizer.from_pretrained("SurtMcGert/advanced-AI-CW-Med-Chat-Bot")
model.config.pad_token_id = tokenizer.pad_token_id
model.config.max_length = 1024

# define the function for applying the chat format
def format_text(message, tokenizer, add_generation_prompt):
    text = tokenizer.apply_chat_template(
        message,
        tokenize=False,
        add_generation_prompt=add_generation_prompt
    )
    return text


# set up chat loop
chat_loop = True
while chat_loop:
    user_input = input("Enter your question: ")
    if(user_input.lower() == "bye"):
        # if the user types "bye" exit the chat
        chat_loop = False
    else:
        # otherwise pass the input into the model
        print(f"user: {user_input}")
        message = [[
                    {"role": "system", "content": "You are a medical professional providing consultation and medical diagnostics."},
                    {"role": "user", "content":f"{user_input}"},
        ]]
        text = list(map(lambda x: format_text(x, tokenizer, True), message))
        encoding = tokenizer(text, padding=True, max_length=1024, return_tensors='pt', add_special_tokens=True)

        # generate a response
        output = model.generate(
            encoding['input_ids'].to(DEVICE),
            max_new_tokens=100, # the max number of new tokens to generate
            early_stopping=True, # stop when unlikely to find better candidates
            repetition_penalty = 1.5, # ads a penalty for repetition
            num_beams=25, # num of beams
            num_beam_groups=1, # num of beam groups
            do_sample=True, # use sampling instead of greedy search
            temperature=0.5, # modulates the next token probabilities
            diversity_penalty=0.0, # ads a penalty for generating unoriginal tokens
            encoder_repetition_penalty=0.99, # ads a penalty for producing tokens that are not in the original input
            no_repeat_ngram_size=5, # all ngrams of this size can only occur once
            # guidance_scale = 1, # Higher guidance scale encourages the model to generate samples that are more closely linked to the input prompt, usually at the expense of poorer quality.
            # length_penalty=-2 # promotes shorter token sequences
        )

        # decode the output
        text_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]

        # parse the text output to just get the models response
        index = text_output.find("<|assistant|>")
        response = text_output[index+13:]
        index = response.find(tokenizer.eos_token)
        if(index > -1):
            response = response[:index]

        # print out the final response
        print(f"assistant: {response}")



# clear memory once the chat has ended
model = None
base_model = None
trainer = None
evaluator = None
torch.cuda.empty_cache()
gc.collect()

user: I am a 39-year-old woman. I have mild pain in the left side of the chest (below the neck and the breast) and then sensation in the upper back for four days. It comes and goes. Sometimes it goes to the right side of the chest also. I had my ECG and blood test 6 months ago. ECG and blood sugar were normal. No hypertension but hemoglobin was 10 and Vitamin B 12 was below average. What can I do as in lockdown it is not possible to see the doctor as a person. Please help.


C:\Users\harry\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


assistant: Hi. I have gone through your history and examination (attachment removed to protect patient identity). You have mentioned that you have pain in left side of chest below the neck and sensation in upper back. You have also mentioned that there is no hypertension and hemoglobin is 10 and vitamin B12 is below average. You have mentioned that your ECG is normal and your blood sugar is normal. I would like to know more about your symptoms.


0